# Семинар 7. Завершение ФП и разговор про классы

На этом семинаре мы закончим обсуждение элементов функционального программирования и немного поговорим про классы.

Функциональные элементы

1. Итераторы
2. Генераторы
3. Функции для ФП; functools
4. Обратная сторона лямбды

Классы

1. Базовые понятия
2. `__dunder__` методы
3. Атрибуты

### Итераторы

Вопрос: в чем разница между `iterable` и `iterator`?

In [ ]:
print(tuple(range(10)))

Задание: напишите проверку на то, является ли объект перебираемым. Вам помогут функция `iter()`, а также обработчик ошибок

In [ ]:
arr = ['i', 'love', 'working', 'with', 'Python']
b = 45.7

###YOUR CODE###

Итератор - объект, который отслеживат свое местонахождение в итерируемым объекте, и по запросу выдает следующее значение. Можно создать итератор по списку, и далее вызывать `next(it)`, пока не получим исключение `StopIteration`

In [ ]:
it = iter([2, 3, 5])

Чтобы вручную реализовать итерацию при помощи итератора: 

• Класс iterable объекта должен содержать метод  `__iter__` для создания и возрата нового итератора

• Класс итератора должен содержать два метода:<br> 
• метод `__init__` , который принимает `iterable` в качестве аргумента и производит остальные необходимые при инициализации вычисления;<br>
• метод `__next__` чтобы найти или вычислить следующее значение. Когда возвращать больше нечего, должен выбрасывать исключение `StopIteration`.

In [ ]:
class MyList(): 
    def __init__(self, ls): 
        self.ls = ls 
        
    def __iter__(self): 
        return Reverser(self.ls) 

class Reverser(): 
    def __init__(self, ls): 
        self.ls = ls 
        self.index = len(self.ls) 
    
    def __next__(self): 
        self.index = self.index - 1
        if self.index >= 0: 
            return self.ls[self.index] 
        raise StopIteration

Как мы видим, итератор использует внутренние переменные для отслеживания своей позиции относительно объекта, по которому мы итерируемся. Поэтому он никак не модифицирует этот объект, и, таким образом, на объекте может быть запущено сразу несколько итераторов параллельно.

Цикле `for` может быть использован с любым итератором. В цикле `for` исключение `StopIteration` не приводит к ошибке, а просто приводит к выходу из цикла.

In [ ]:
ls = MyList([1, 2, 3, 4]) 

for e in ls: 
    print(e)

Задание со звездочкой: реализуйте цикл `for` при помощи итератора. Подсказка: обработчик ошибок вам также в помощь тут

In [ ]:
###YOUR CODE###

Итератор не может быть "переиспользован", или же как-то "перезагружен" - после того, как он выбросит `StopIteration`, мы можем только создать новый.

### Генераторы

Генераторы похожи на итераторы. Они также генерируют значения по одному, однако при этом не обязательно привязаны к какому-то объекту.

Генератор можно задать почти как списочное включение, где вместо квадратных скобок будут круглые, и использовать во всех тех же случаях, в которых можно использовать итератор:

In [ ]:
word = 'generator'
gen = (c for c in word if c in 'aeiou')
for i in gen:
    print(i, end=' ')

Как и итератор, после полного цикла он принимает пустое значение и больше ничего не вернет.

In [ ]:
for i in gen:
    print(i, end=' ')

Смысл генератора чуть можно лучше прояснить, если попробовать написать функцию, которая ведет себя как генераторы. В таких функциях вместо `return` мы используем `yield`

In [ ]:
def powers_of_two():
    n = 2
    for i in range(0, 5):
        yield n
        n *= 2

При вызове функции возвращается не число, а генератор, который мы затем можем использовать

In [ ]:
gen = powers_of_two()
for n in gen:
    print(n)

Генерируем следующее значение, и обрабатываем исключение

In [ ]:
gen = powers_of_two()
while True:
    try:
        print(next(gen))
    except StopIteration:
        break

Что тут происходит:<br>
• Вызов `gen = powers_of_two()` возвращает генератор и кладет в переменную `gen`.<br>
• Первый вызов `next(gen)` исполнит код генератора до `yield` и вернет необходимое значение, как обычный `return`. Однако вдобавок генератор запомнит свое состояние.<br>
• Следующий вызов `next(gen)` вернется к исполнению кода в генераторе с предыдущего состояния, то есть сразу после `yield`. Все значения локальных переменных будут восстановлены - как будто бы `yield` не происходил. В нашем примере, цикл `for` продолжит исполнение (кстати, `yield` можно использовать более одного раза в рамках одной функции).<br>
• В конце - все так же `StopIteration`.

#### Генераторы и память

In [ ]:
import sys

# проверяем расход памяти
def memory_size(_, code):
    size = sys.getsizeof(code)
    return f'{_}: allocated memory is {size} bytes'

print(memory_size('generator', (num**2 for num in range(10000))))
print(memory_size('list comprehension', [num**2 for num in range(10000)]))

Генераторы позволяют нам работать с большими датасетами с минимальными затратами памяти.

### Прочие функциональные инструменты

Питон располагает рядом встроенных инструментов:

`map(function, iterable)` применяет функцию к каждому элементу в перечисляемом объекте и возвращает перечисляемый объект `map`

In [ ]:
print(map(lambda x: x + "bzz!", ["I think", "I'm good"]))
list(map(lambda x: x + "bzz!", ["I think", "I'm good"]))

`filter(function or None, iterable)` отбирает элементы из перечисляемого объекта на основании результата применения к ним функции, и возвращает перечисляемый объект `filter`

In [ ]:
print(filter(lambda x: x.startswith("I "), ["I think", "I'm good"]))
list(filter(lambda x: x.startswith("I "), ["I think", "I'm good"]))

`zip(iter1 [,iter2 [...]])` принимает на вход последовательности одной длины, и поэлементно собирает их в последовательность кортежей. Полезно, например, если нам нужно объединить список ключей и список значений в словарь. Возвращает перечисляемый объект `zip`

In [ ]:
keys = ["foobar", "barzz", "ba!"]
print(zip(keys, map(len, keys)))
print(list(zip(keys, map(len, keys))))
print(dict(zip(keys, map(len, keys))))

При помощи `zip` можно проитерироваться по двум спискам сразу. Задание: попробуйте реализовать это

In [ ]:
first_names = ['George','Keith', 'Art']
last_names = ['Luke', 'Dell','Funnel']

###YOUR CODE###

`functools.reduce(binaryFunction, iterable)` применяет бинарную функцию к первым двум элементам перечисляемого объекта. Затем операция последовательно повторяется по отношению к результату применения функции и следующему элементу, пока перечисляемый объект не свернется в одно результирующее значение. (NB: `reduce` необходимо импортировать из библиотеки `functools`).

In [ ]:
my_list = [3, 1, 4, 1, 6]
from functools import reduce
reduce(lambda x, y: x + y, my_list)

Задание: реализуйте факториал в одну строчку кода (hint: может пригодиться `range()`)

In [ ]:
from functools import reduce
num = 9

###YOUR CODE HERE###

`any(iterable)` и `all(iterable)` возвращают булево значение в зависимости от значений на элементах перечисляегого объекта. Они эквивалентны следующему:

In [ ]:
def all(iterable):
    for x in iterable:
        if not x:
            return False
    return True

In [ ]:
def any(iterable):
    for x in iterable:
        if x:
            return True
    return False

Их удобно применять в том случае, если нужно проверить, соответствуют ли все / хоть один элемент в перечисляемом объекте условию:

In [ ]:
mylist = [0, 1, 3, -1]
if all(map(lambda x: x > 0, mylist)):
    print("All items are greater than 0")
if any(map(lambda x: x > 0, mylist)):
    print("At least one item is greater than 0")

#### Вернемся к списочным включениям в этом контексте

Как мы выяснили, списочные включения - это отличный способ задания списков. Стоит отметить, что списки, которые заданы при помощи включений, ни в чем не опираются на состояние программы, и потому вписываются в парадигму функционального программирования (и именно в силу этого является столь элегантным и легким к восприятию приемом).

Задание: напишите эквивалент `filter` и `map` при помощи списочного включения:

`map`

In [ ]:
###YOUR CODE###

`filter`

In [ ]:
###YOUR CODE###

### Обратная сторона лямбды

Подоплека: когда мы работаем со списками, часто встает задача найти первый элемент, удовлетворяющий определенному условию. Обычно эта задача решается при помощи конструкции наподобие следующей:

In [ ]:
def first_positive_number(numbers):
    for n in numbers:
        if n > 0:
    return n

В функциональном стиле можно переписать так:

In [ ]:
def first(predicate, items):
    for item in items:
        if predicate(item):
    return item

first(lambda x: x > 0, [-1, 0, 1, 2])

In [ ]:
# Less efficient
list(filter(lambda x: x > 0, [-1, 0, 1, 2]))[0]
# Efficient
next(filter(lambda x: x > 0, [-1, 0, 1, 2]))

Вопрос: какую вы видите проблему с первым вариантом?

Можно немного упростить себе жизнь, воспользовавшись небольшой полезной библиотекой first:

In [ ]:
from first import first

a = first([0, False, None, [], (), 42])
b = first([-1, 0, 1, 2])
c = first([-1, 0, 1, 2], key=lambda x: x > 0)
print(a, b, c)

Здесь мы используем именованный аргумент `key` с тем, чтобы передать в него функцию, которая поочередно оценивает каждый аргумент на соответствие условию.

Как можно увидеть, в семинаре мы повсеместно используем для подобных случаев `lambda`. Вообще говоря, `lambda` была включена в Python в первую очередь для удобства использования функциональных `map()` и `filter()`, который в противном случае потребовали бы определять новую функцию каждый раз когда нужно проверить какое-то новое условие.

In [ ]:
import operator
from first import first

def greater_than_zero(number):
    return number > 0

first([-1, 0, 1, 2], key=greater_than_zero)

Код выше работает аналогично предыдущей ячейке, но он гораздо более громоздкий. Если, например, нам понадобится найти первое число в последовательности, которое превосходит, скажем, 42, то нам бы пришлось определять соответствующую отдельную функцию вместо того, чтобы определить ее в одной строке с вызовом `first`.

Несмотря на всю полезность в подобных ситуациях, `lambda` не лишена проблем. Самое очевидное, мы не можем таким образом передать в ключ функцию длиной более одной строчки кода. Однако означает ли это, что в этой ситуации у нас нет другого выхода, кроме как определять отдельную функцию? Не совсем.

`functools.partial` предоставляет альтернативу лямбда-выражению, причем даже более гибкую. Эта конструкция позволяет создать функцию-обертку с интересной особенностью: вместо того, чтобы менять поведение функции, мы взамен меняем аргументы, которые она получает на вход. Например:

In [ ]:
from functools import partial
from first import first

def greater_than(number, min=0):
    return number > min

first([-1, 0, 1, 2], key=partial(greater_than, min=42))

Теперь наша функция `greater_than` по умолчанию работает как прежняя, но вдобавок мы можем указать значение, с которым мы сравниваем передаваемый в нее аргумент. В данном случае, мы передаем в `functools.partial` нашу исходную функцию и значение, которым мы переопределим `min`, и в результате получим новую функцию, которая сравнивает числа на входе с 42, ровно как мы и хотели бы. Другими словами, мы можем задать функцию, и затем кастомизировать ее при помощи `functools.partial` так, как нам необходимо в данной ситуации.

Строго говоря, в данном конкретном случае наша запись все еще избыточна, ведь все, что мы делаем, это сравнение двух чисел. В Питон присутствует специальный модуль `operator` для подобного рода простых операций:

In [ ]:
import operator
from functools import partial
from first import first

first([-1, 0, 1, 2], key=partial(operator.le, 0))

Как мы можем убедиться, `functools.partial` работает и с позиционными аргументами тоже. В данном примере `operator.le(a, b)` принимает на вход два числа и возвращает булево значение в зависимости от того, больше или равно первое второму или нет. Ноль, который мы передаем в `functools.partial`, уходит в переменную `a`, в то время как то, что уходит в функцию, которую мы получаем на выходе из `functools.partial`, уходит в `b`. Таким образом, используя `le` (а не `ge`, как могло бы показаться), наш пример работает должным образом без необходимости в лямбда-выражении и задании каких-либо дополнительных функций.

`functools.partial` особенно полезна в качестве замены `lambda` - которую, к слову сказать, даже планировали убрать из третьей версии Пайтон! - и считается предпочтительной альтернативой. Лямбда-выражения являются некоторой аномалией ввиду ограничения на длину в одну стоку. С другой стороны, `functools.partial` предоставляет удобную обертку вокруг исходной функции.

In [ ]:
from functools import partial
binStrToInt = partial(int, base=2)
print(binStrToInt('10010'))

Вообще говоря, модуль `itertools` в составе Python Standard Library содержит целый ряд полезных функций, которые неплохо держать в уме. Очень часто можно встретить примеры, когда разработчики прописывают свои версии данных функций... называя вещи своими именами, изобретают велосипед, когда есть отличные готовые реализации:

• `chain(*iterables)` итерация по элементам перечисляемого объекта без явного построения промежуточного списка всех элементов<br>
• `combinations(iterable, r)` генерирует все комбинации длины `r` из данного перечисляемого объекта<br>
• `compress(data, selectors)` применяет булеву маску из `selectors` к данным и возвращает только те значения из них, где соответствующие элемент селектора истинен<br>
• `count(start, step)` генерирует бесконечную последовательность значений, начиная со `start` и увеличиваясь на `step` на каждом вызове<br>
• `cycle(iterable)` циклически перебирает элементы в перечисляемом объекте<br>
• `dropwhile(predicate, iterable)` отфильтровывает элементы перечисляемого объекта с начала и до момента когда предикат оценится как ложный<br>
• `groupby(iterable, keyfunc)` создает итератор, который группирует элементы по результату, который возвращает на них функция `keyfunc`<br>
• `permutations(iterable[, r])` возвращает последовательные перестановки элементов перечисляемого объекта длины `r`<br>
• `product(*iterables)` возвращает перечисляемый объект декартова произведения перечисляемых объектов без задействования вложенных циклов<br>
• `takewhile(predicate, iterable)` возвращает элементы перечисляемого объекта с начала и до момента когда предикат оценится как ложный<br>

Наибольшую мощь эти функции приобретают в комбинации с модулем `operator`; сочетание `itertools` и `operator` может заменить собой `lambda` практически во всех ситуациях:

In [ ]:
import itertools

a = [{'foo': 'bar'}, {'foo': 'bar', 'x': 42}, {'foo': 'baz', 'y': 43}]

import operator

print(list(itertools.groupby(a, operator.itemgetter('foo'))))
[(key, list(group)) for key, group in list(itertools.groupby(a, operator.itemgetter('foo')))]

В данном случае можно было бы воспользоваться конструкцией `lambda x: x['foo']`, однако использование `operator` позволяет совсем отказаться от лямбда-выражения.

# Элементы ООП

### Базовые сведения про классы

Классы - ключевая концепция в ООП чтобы определять новые типы объектов. Классы позволяют нам абстрагироваться от имплементации объекта и манипулировать ими более верхнеуровнево через внешний интерфейс.

In [ ]:
class MyClass:
    pass

In [ ]:
my_object = MyClass()
my_object

В классе мы можем определить связанные с ним методы, определяющие поведение объекта и/или производящие операции над его свойствами. Первым параметром метода всегда является переменная, ссылающаяся на сам класс. По соглашению (и не более того) ее именуют `self`.

In [ ]:
class MyClass:
    def my_method(self):
        print("Hello from my_method!")

In [ ]:
my_object.my_method()

Разумеется, методы могут иметь и другие параметры и возвращать значения, по аналогии с функциями

In [ ]:
class MyClass:
    def add(self, a, b):
        return a + b

my_object = MyClass()
result = my_object.add(2, 3)
result

И вообще, в каком-то смысле методы и есть функции, привязанные к пространству имен класса. В функциональном смысле нет никакой разницы между

In [ ]:
str.join("", ('a', 'b'))

и

In [ ]:
"".join(('a', 'b'))

Наконец, мы можем наследовать подкласс от надкласса (`superclass`).

In [ ]:
class YourClass:
    def subtrace(self, a, b):
        return a - b

class MyClass(YourClass):
    pass

my_object = MyClass()
result = my_object.subtract(2, 3)
result

Вообще, можно заменить, что вызывая `type()` от многих объектов, зачастую возвращается класс

In [ ]:
x = "Hello!"
type(x)

Все эти встроенные в Питон типы имеют свои методы - к примеру, в классе `str` определен `join()`, который принимает в качестве аргумента вызывающий объект, т.е. строку, как `self`, и набор прочих строк, которые соединяются между собой с `self` в качестве разделителя.

In [ ]:
" ".join(('a', 'b', 'c'))

Принимая во внимание данный факт, мы также можем вызвать метод напрямую от класса, задав в таком случае аргумент `self` явно:

In [ ]:
str.join(" ", ('a', 'b', 'c'))

### `__dunder__` методы

“dunder” (“double underscore”) еще называют “magic методами” - это специальные методы со специальным значением, определяющие особое поведение объектов, например, `__init__` или `__len__`. Пока остановимся подробнее над
`__init__`.

`__init__` вызывается автоматически, когда создается объект, и служит для инициализации атрибутов объекта. Важно: `__init__` **не** является конструктором, это инициализатор, который вызывается автоматически **из** конструктора (когда, собственно, конструируется новый объект).

После того, как мы определили `__init__`, мы можем создать экземпляр класса и передать в него аргументы, посредством вызова класса с соответствующими параметрами. При этом аргументы будут переданы в вызов метода `__init__`, и будут присвоены экземпляру класса `self` как атрибуты.

In [ ]:
class MyClass:
    def __init__(self, my_value):
        self.my_value = my_value

my_object = MyClass(5)
my_object.my_value

In [ ]:
class MyClass:
    def __init__(self, my_value):
        self.my_value = my_value
        
    def my_method(a):
        print("Hello from my_method!")

my_object = MyClass(5)
my_object.my_value
my_object.my_method()

In [ ]:
class MyClass:
    def __init__(self, my_value):
        self.my_value = my_value
        
    def my_method(a):
        print(a.my_value)
        print("Hello from my_method!")

my_object = MyClass(5)
my_object.my_value
my_object.my_method()

### Собственно, атрибуты

Атрибуты - это переменные, ассоциированные с классом или экземпляром класса. В них хранится состояние объекта, и доступ к ним осуществляется через `.`

Самый распространенный способ задания атрибутов - как переменные экземпляра класса внутри любого метода класса через `self` (или то, что его заменяет в случае отхода от соглашения). Именно это мы сделали в примере с `__init__`.

Еще примеры:

In [ ]:
my_object = MyClass(5)
my_object.my_value

In [ ]:
my_object.my_value = 10
my_object.my_value

Атрибуты класса - переменные, определенные на уровне класса, а не экземпляра класса. Доступ к ним возможен как через наименование экземепляра класса, так и через наименование самого класса.

In [ ]:
class MyClass:
    class_attribute = "I am a class attribute."

obj1 = MyClass()
obj2 = MyClass()

obj1.class_attribute

In [ ]:
obj2.class_attribute

In [ ]:
MyClass.class_attribute

В случае использования изменяемого типа в качестве атрибута класса важно не попасть в хорошо знакомую нам ловушку:

In [ ]:
class One:
    items = []

a = One()
b = One()

a.items.append(1)
b.items